In [46]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

Продовжуємо працювати з датасетом titanic.csv
Завантажуємо, проводимо попередній аналіз даних (лр 2).
Будуємо 3 класифікатори на вибір, визначаємо оптимальні параметри
(оптимізатор – на вибір). Вибираємо найкращу модель. Виводимо
характеристики, метрики.
Робимо прогноз для 10 випадків.

In [35]:
from google.colab import files
import pandas as pd
# Завантаження файлу
uploaded = files.upload()

df = pd.read_csv('titanic.csv')
df

Saving titanic.csv to titanic.csv


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,0,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,0,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,0,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    int64  
 2   Pclass       418 non-null    int64  
 3   Name         418 non-null    object 
 4   Sex          418 non-null    object 
 5   Age          332 non-null    float64
 6   SibSp        418 non-null    int64  
 7   Parch        418 non-null    int64  
 8   Ticket       418 non-null    object 
 9   Fare         417 non-null    float64
 10  Cabin        91 non-null     object 
 11  Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB


Підготовка даних

In [58]:
data = pd.read_csv('titanic.csv')
data_clean = data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
data_clean['Age'].fillna(data_clean['Age'].median(), inplace=True)
data_clean['Fare'].fillna(data_clean['Fare'].median(), inplace=True)
data_clean['Embarked'].fillna(data_clean['Embarked'].mode()[0], inplace=True)

data_clean = pd.get_dummies(data_clean, columns=['Sex', 'Embarked'], drop_first=True)

print("Після обробки:")
print(data_clean.info())
print("\nПерші 5 рядків після обробки:")
print(data_clean.head())

Після обробки:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    418 non-null    int64  
 1   Pclass      418 non-null    int64  
 2   Age         418 non-null    float64
 3   SibSp       418 non-null    int64  
 4   Parch       418 non-null    int64  
 5   Fare        418 non-null    float64
 6   Sex_male    418 non-null    bool   
 7   Embarked_Q  418 non-null    bool   
 8   Embarked_S  418 non-null    bool   
dtypes: bool(3), float64(2), int64(4)
memory usage: 20.9 KB
None

Перші 5 рядків після обробки:
   Survived  Pclass   Age  SibSp  Parch     Fare  Sex_male  Embarked_Q  \
0         0       3  34.5      0      0   7.8292      True        True   
1         1       3  47.0      1      0   7.0000     False       False   
2         0       2  62.0      0      0   9.6875      True        True   
3         0       3  27.0      0      

/tmp/ipython-input-3195650747.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_clean['Age'].fillna(data_clean['Age'].median(), inplace=True)
/tmp/ipython-input-3195650747.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inpl

Модель 1: Логістична регресія

In [39]:
logreg = LogisticRegression()
param_grid_logreg = {'C': [0.1, 1, 10], 'solver': ['liblinear', 'lbfgs']}
grid_logreg = GridSearchCV(logreg, param_grid_logreg, cv=5)
grid_logreg.fit(X_train, y_train)
y_pred_logreg = grid_logreg.predict(X_test)

print("\nЛогістична регресія:")
print("Точність:", accuracy_score(y_test, y_pred_logreg))
print("Найкращі параметри:", grid_logreg.best_params_)


Логістична регресія:
Точність: 0.9035087719298246
Найкращі параметри: {'C': 10, 'solver': 'liblinear'}


Модель 2: Дерево рішень

In [41]:
tree = DecisionTreeClassifier()
param_grid_tree = {'max_depth': [3, 5, 7], 'min_samples_split': [2, 5, 10]}
grid_tree = GridSearchCV(tree, param_grid_tree, cv=5)
grid_tree.fit(X_train, y_train)
y_pred_tree = grid_tree.predict(X_test)

print("\nДерево рішень:")
print("Точність:", accuracy_score(y_test, y_pred_tree))
print("Найкращі параметри:", grid_tree.best_params_)


Дерево рішень:
Точність: 0.9210526315789473
Найкращі параметри: {'max_depth': 3, 'min_samples_split': 2}


Модель 3: Випадковий ліс

In [42]:
forest = RandomForestClassifier()
param_grid_forest = {'n_estimators': [50, 100], 'max_depth': [5, 7]}
grid_forest = GridSearchCV(forest, param_grid_forest, cv=5)
grid_forest.fit(X_train, y_train)
y_pred_forest = grid_forest.predict(X_test)

print("\nВипадковий ліс:")
print("Точність:", accuracy_score(y_test, y_pred_forest))
print("Найкращі параметри:", grid_forest.best_params_)


Випадковий ліс:
Точність: 0.9298245614035088
Найкращі параметри: {'max_depth': 7, 'n_estimators': 100}


Порівняння моделей

In [43]:
accuracies = {
    'Логістична регресія': accuracy_score(y_test, y_pred_logreg),
    'Дерево рішень': accuracy_score(y_test, y_pred_tree),
    'Випадковий ліс': accuracy_score(y_test, y_pred_forest)
}

best_model_name = max(accuracies, key=accuracies.get)
print(f"\nНайкраща модель: {best_model_name} з точністю {accuracies[best_model_name]:.4f}")


Найкраща модель: Випадковий ліс з точністю 0.9298


Прогноз для 10 випадків

In [73]:
-

SyntaxError: invalid syntax (ipython-input-476313318.py, line 1)